# Video demo

This notebook is for testing my pipeline.

I am using the 1.m4v video from Eleni.

**TODO** 
- Fix it so that the output video is stored in runs_uwyolo in a new folder 1m4_video_demo_results





## Data setup

Download and put the 1.mv4 file inside datafolder. I have access to the videofile

### Sterero -> Mono

In [1]:
import cv2
import os

# ==== INPUT / OUTPUT PATHS ====
input_path = "../data/1.m4v"          # your original stereo video
output_path = "../data/stereo_left_half.mp4"     # or .m4v if you prefer

# ==== READ VIDEO ====
cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise IOError(f"❌ Cannot open video: {input_path}")

# ==== GET VIDEO INFO ====
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
half_width = width // 2
print(f"🎥 Input: {width}x{height} @ {fps}fps → cropping to {half_width}x{height}")

# ==== SET UP OUTPUT WRITER ====
fourcc = cv2.VideoWriter_fourcc(*'mp4v')   # or 'avc1' for m4v output
out = cv2.VideoWriter(output_path, fourcc, fps, (half_width, height))

# ==== FRAME LOOP ====
while True:
    ret, frame = cap.read()
    if not ret:
        break
    left = frame[:, :half_width]     # crop left half (all rows, left columns)
    out.write(left)

cap.release()
out.release()
print(f"✅ Saved cropped video to: {output_path}")


🎥 Input: 3840x1200 @ 25.0fps → cropping to 1920x1200
✅ Saved cropped video to: ../data/stereo_left_half.mp4


### Make video last X seconds

In [1]:
import cv2

# ==== CONFIG ====
input_path = "../data/stereo_left_half.mp4"
output_path = "../data/stereo_left_half_60sec.mp4"
clip_seconds = 60   # 🔁 change this to however many seconds you want to keep

# ==== OPEN VIDEO ====
cap = cv2.VideoCapture(input_path)
if not cap.isOpened():
    raise IOError(f"❌ Cannot open video: {input_path}")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# ==== COMPUTE END FRAME ====
keep_frames = int(min(frame_count, clip_seconds * fps))
print(f"🎞 Total {duration:.1f}s ({frame_count} frames) → keeping first {clip_seconds}s ({keep_frames} frames)")

# ==== SET UP OUTPUT ====
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# ==== FRAME LOOP ====
frame_idx = 0
while frame_idx < keep_frames:
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"✅ Saved first {clip_seconds}s to: {output_path}")


🎞 Total 720.0s (18000 frames) → keeping first 60s (1500 frames)
✅ Saved first 60s to: ../data/stereo_left_half_60sec.mp4


## Run UW_yolo on Monovideo

In [13]:
from ultralytics import YOLO
import subprocess

model = YOLO("../runs_uwyolo/ruod_0873_sgd300_4gpu/weights/best.pt")

results = model.predict(
    source="../data/stereo_left_half_60sec.mp4",   
    imgsz=640,
    conf=0.25,
    iou=0.7,
    save=True,
    project="runs_uwyolo",
    name="video_demo",
    exist_ok=True,
    save_txt=False,
    save_conf=False
)

# results is a list; use the first element
save_dir = results[0].save_dir
print("✅ Inference done. Results saved in:", save_dir)





WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1500) /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/utils/../data/stereo_left_half_60sec.mp4: 416x640 5 fishs, 12.6ms
video 1/1 (frame 2/1500) /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/utils/../data/stereo_left_half_60sec.mp4: 416x640 5 fishs, 13.0ms
video 1/1 (frame 3/1500) /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/utils/../data/stereo_left_half_60sec.mp4: 416x640 7 fishs, 11.8ms
video 1/1 (frame 4/1500) /cluster/home/henrban/SOLA

### .avi -> .mp4


pip install imageio imageio-ffmpeg

pip install imageio[pyav]


In [15]:
import cv2
from pathlib import Path

# ==== CONFIG ====
input_path = Path("./runs_uwyolo/video_demo/stereo_left_half_60sec.avi")

output_path = input_path.with_suffix(".mp4")   # output with same name but .mp4

# ==== OPEN INPUT ====
cap = cv2.VideoCapture(str(input_path))
if not cap.isOpened():
    raise IOError(f"❌ Cannot open video: {input_path}")

fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")   # or 'avc1' for H.264 codec

# ==== OPEN OUTPUT ====
out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

print(f"🎞️ Converting {input_path.name} → {output_path.name} ({width}x{height} @ {fps:.2f}fps) ...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)

cap.release()
out.release()

print(f"✅ Done! Saved to: {output_path.resolve()}")


🎞️ Converting stereo_left_half_60sec.avi → stereo_left_half_60sec.mp4 (1920x1200 @ 25.00fps) ...
✅ Done! Saved to: /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/utils/runs_uwyolo/video_demo/stereo_left_half_60sec.mp4
